In [1]:
import os
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# ---------- Configuration ----------
MAX_WORKERS = 1
EXCLUDE_PREFIXES = [
    '__pycache__', 
    '.ipynb_checkpoints', 
    'base', 
    'completed runs', 
    'to do', 
    'analysis'
]

# Lock for thread-safe printing
print_lock = Lock()

# ---------- Core Helpers ----------
def run_notebook(folder: str, notebook_name: str) -> str:
    """
    Execute a Jupyter notebook in place using nbconvert.
    Returns a status string with execution time.
    All printing inside this function is thread-safe.
    """
    notebook_path = os.path.join(folder, notebook_name)
    if not os.path.exists(notebook_path):
        with print_lock:
            print(f"SKIP: {notebook_name} not found in {folder}", flush=True)
        return f"SKIP: {notebook_name} not found in {folder}"

    with print_lock:
        print(f"🔹 Running {notebook_path}...", flush=True)

    start_time = time.time()
    try:
        subprocess.run(
            [
                'jupyter', 'nbconvert', '--to', 'notebook',
                '--execute', notebook_path, '--inplace',
                '--ExecutePreprocessor.timeout=-1'
            ],
            check=True,
            capture_output=True,
            text=True
        )
        elapsed = time.time() - start_time
        return f"✅ SUCCESS: {folder}/{notebook_name} (took {elapsed:.2f}s)"
    except subprocess.CalledProcessError as e:
        elapsed = time.time() - start_time
        stderr_clean = e.stderr.strip()
        return f"❌ ERROR: {folder}/{notebook_name} - {stderr_clean} (took {elapsed:.2f}s)"

def find_folders(base_dir: str = '.') -> list[str]:
    """Return a list of folders to process, excluding certain prefixes."""
    return [
        d for d in os.listdir(base_dir)
        if os.path.isdir(d) and not any(d.startswith(prefix) for prefix in EXCLUDE_PREFIXES)
    ]

def run_in_parallel(folders: list[str], notebook_names: list[str]):
    """Run one or more notebooks across all folders in parallel, with thread-safe printing."""
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [
            executor.submit(run_notebook, folder, nb)
            for folder in folders
            for nb in notebook_names
        ]

        for future in as_completed(futures):
            msg = future.result().strip()
            with print_lock:
                print(msg)
    with print_lock:
        print("✅ All done!")

# ---------- Specialized Variants ----------
def run_random_perturbs_parallel():
    """Run Random Perturbs.ipynb across all model_* folders."""
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders for Random Perturbs.")
    run_in_parallel(folders, ["Random Perturbs.ipynb"])

def run_volume_parallel():
    """Run all notebooks containing 'Volume Cutoff' or 'Volume Estimation' across folders."""
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders for Volume notebooks.")

    notebook_names = []
    if folders:
        for nb in os.listdir(folders[0]):
            if nb.endswith(".ipynb") and ("Volume Cutoff" in nb or "Volume Estimation" in nb):
                notebook_names.append(nb)

    if not notebook_names:
        with print_lock:
            print("⚠️ No Volume notebooks found in reference folder.")
        return

    with print_lock:
        print(f"Will run these notebooks: {notebook_names}")
    run_in_parallel(folders, notebook_names)

def run_nb_parallel():
    """Generic function for ad-hoc notebook runs (prompt user)."""
    notebook_name = input("Enter notebook name (without .ipynb): ").strip() + ".ipynb"
    folders = find_folders()
    with print_lock:
        
        print(f"Found {len(folders)} folders to process.")
    run_in_parallel(folders, [notebook_name])

run_random_perturbs_parallel()

Found 50 folders for Random Perturbs.
🔹 Running model_40_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_40_data_10/Random Perturbs.ipynb (took 2367.40s)
🔹 Running model_0_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_0_data_10/Random Perturbs.ipynb (took 2392.47s)
🔹 Running model_22_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_22_data_10/Random Perturbs.ipynb (took 2443.35s)
🔹 Running model_32_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_32_data_10/Random Perturbs.ipynb (took 2437.27s)
🔹 Running model_14_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_14_data_10/Random Perturbs.ipynb (took 2382.96s)
🔹 Running model_49_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_49_data_10/Random Perturbs.ipynb (took 2405.11s)
🔹 Running model_9_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_9_data_10/Random Perturbs.ipynb (took 2445.08s)
🔹 Running model_15_data_10/Random Perturbs.ipynb...
✅ SUCCESS: model_15_data_10/Random Perturbs.ipynb (took 2442.43s)
🔹 Running model_8_data